# Análise de reviews de produtos usando NLP

O presente trabalho pretende analisar os textos de revisão de produtos da base de dados obtida na plataforma Kaggle. Para isso serão usadas bibliotecas Python de processamento de linguagem natural já conhecidas como Spacy, Sci-kit Learn e.

https://www.kaggle.com/olistbr/brazilian-ecommerce

Objetivos:
- Encontrar caracaterísticas que distinguam textos de reviews com notas altas e com notas baixas.
- Analisar textos com análise de sentimentos e identificar textos muito negativos, mas que possuam notas altas.

In [1]:
import pandas as pd
import sklearn
import spacy

In [7]:
e_set = pd.read_csv("./brazilian-ecommerce/olist_public_dataset_v2.csv")
e_set.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01.000000,2017-02-01 02:41:21.549551,2017-03-15 00:00:00.000000,2017-02-06 11:04:24.154259,...,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07 00:00:00.000000,2017-02-09 02:37:37+00:00
1,59af46052a799e80e2f0c665c587731d,delivered,15.00,15.10,1,1,2017-09-09 19:52:54.000000,2017-09-10 20:03:31.535281,2017-10-02 00:00:00.000000,2017-09-13 20:17:41.296915,...,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,NaN,entrega em 2 dias produto c boa qualidade otim...,2017-09-14 00:00:00.000000,2017-09-15 03:43:47+00:00
2,a3e6136894621db402a772c6bc72a12a,delivered,238.90,18.00,1,1,2017-01-30 17:00:09.000000,2017-01-30 17:31:25.438253,2017-03-07 00:00:00.000000,2017-02-06 15:43:04.758566,...,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,NaN,produto veio antes do prazo informado muito ob...,2017-02-07 00:00:00.000000,2017-02-10 14:18:53+00:00
3,b675ea5a618922f6e679e30531b8957b,delivered,29.99,18.23,1,1,2018-03-11 18:18:36.000000,2018-03-11 18:30:37.931962,2018-04-03 00:00:00.000000,2018-04-03 20:36:43.778451,...,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,NaN,NaN,2018-04-04 00:00:00.000000,2018-04-05 02:52:31+00:00
4,195a8be6794c487fe6cfbb97b7c61902,delivered,295.99,47.65,1,1,2017-04-20 08:01:08.000000,2017-04-25 08:05:40.405383,2017-05-24 00:00:00.000000,2017-05-04 18:47:45.721758,...,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,NaN,NaN,2017-05-05 00:00:00.000000,2017-05-08 15:20:18+00:00


In [11]:
e_work_set = e_set[['order_products_value','review_score', 'review_comment_message']].copy()
e_work_set.head()

,order_products_value,review_score,review_comment_message
0,349.90,5,NaN
1,15.00,5,entrega em 2 dias produto c boa qualidade otim...
2,238.90,5,produto veio antes do prazo informado muito ob...
3,29.99,4,NaN
4,295.99,5,NaN


In [17]:

e_set_rev_1 = e_work_set[e_work_set['review_score'] == 1]
e_set_rev_1 = e_set_rev_1.dropna()
e_set_rev_1.head()
print(len(e_set_rev_1))

9304


# Lista de palavras positivas e negativas

Para a análise de sentimento, é usado uma lista também do site Kaggle.
https://www.kaggle.com/rtatman/sentiment-lexicons-for-81-languages

# Filtro de Stopwords

As stopwords, palavras que não possuem sentido significativo para a sentença são retiradas através da corpora disponibilizada pela bibliteca NLTK.

In [24]:
from nltk import corpus

In [27]:
lista = corpus.stopwords.words('portuguese')

de
a
o
que
e
do
da
em
um
para
com
não
uma
os
no
se
na
por
mais
as
dos
como
mas
ao
ele
das
à
seu
sua
ou
quando
muito
nos
já
eu
também
só
pelo
pela
até
isso
ela
entre
depois
sem
mesmo
aos
seus
quem
nas
me
esse
eles
você
essa
num
nem
suas
meu
às
minha
numa
pelos
elas
qual
nós
lhe
deles
essas
esses
pelas
este
dele
tu
te
vocês
vos
lhes
meus
minhas
teu
tua
teus
tuas
nosso
nossa
nossos
nossas
dela
delas
esta
estes
estas
aquele
aquela
aqueles
aquelas
isto
aquilo
estou
está
estamos
estão
estive
esteve
estivemos
estiveram
estava
estávamos
estavam
estivera
estivéramos
esteja
estejamos
estejam
estivesse
estivéssemos
estivessem
estiver
estivermos
estiverem
hei
há
havemos
hão
houve
houvemos
houveram
houvera
houvéramos
haja
hajamos
hajam
houvesse
houvéssemos
houvessem
houver
houvermos
houverem
houverei
houverá
houveremos
houverão
houveria
houveríamos
houveriam
sou
somos
são
era
éramos
eram
fui
foi
fomos
foram
fora
fôramos
seja
sejamos
sejam
fosse
fôssemos
fossem
for
formos
forem
serei
será
seremos
se